In [62]:
#import all libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import zipfile
import logging

In [63]:
def get_links():
    urls = list(pd.read_csv('books_url.csv')['link'])
    return urls

In [64]:
def get_soup(input_url):
    headers = {
        'User-Agent': 'My User Agent 1.0',
        "Accept-Language": "en-US,en;q=0.5"
    }
    response = requests.get(input_url, headers=headers)
    if response.status_code != 200:
        print("Error in getting link")
        print("response code is : ", response.status_code)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [65]:
def get_fa_title(soup):
    title = soup.select('.product-name strong')[0]
    return title.text


In [66]:
def get_en_title(soup):
    title = soup.select('.product-name-englishname')[0]
    return title.text

In [67]:
def get_price(soup):
    price = soup.select('.price-broken , .col-md-7 .price:nth-child(1)')[0].text
    return int(price.replace(',', ''))  #TODO make the price integer

In [86]:
def get_discount(soup):
    try:
        discount_price = int(soup.select('.col-md-12+ .clearfix .price-special')[0].text.replace(',', ''))
        discount_price = ((get_price(soup) - discount_price) / get_price(soup)) * 100
    except Exception:
        discount_price = 0
        logging.exception("This book has no discount!")
    return discount_price  

In [69]:
def get_score(soup):
    soup = soup.find('div', {'class':'col-md-7'}).find('li', {'class':'pull-left'}).find('div', {'class':'my-rating'})
    soup_str = str(soup)

    match = re.search(r'data-rating="(\d+\.\d+)"', soup_str)
    if match:
        data_rating = match.group(1)
        return data_rating
    

In [70]:
def get_publisher(soup):
    try:
        publisher = soup.select('.prodoct-attribute-items:nth-child(1) a .prodoct-attribute-item')[0].text.strip()
    except Exception:
        publisher = None
        logging.exception("This book has no publisher!")
    return publisher

In [71]:
def get_author(soup):
    try:
        author = soup.select('.prodoct-attribute-items+ .prodoct-attribute-items a .prodoct-attribute-item')[0].text.strip()
    except Exception:
        author = None
        logging.exception("This book has no author!")    
    return author

In [72]:
def get_code(soup):
    code = soup.select('tr:nth-child(1) td+ td')[0]
    return code.text

In [73]:
def get_isbn(soup):
    isbn = soup.select('.product-table .ltr')[0]
    cleaned_isbn = isbn.text.strip()
    cleaned_isbn = re.sub('[^0-9-]', '', cleaned_isbn)
    return cleaned_isbn

In [74]:
def get_pages(soup):
    pages = soup.select('tr:nth-child(5) .rtl')[0]
    cleaned_pages = pages.text.strip()
    cleaned_pages = re.sub('[^0-9-]', '', cleaned_pages)
    return cleaned_pages

In [75]:
def get_per_date(soup):
    try:
        date = soup.select('.product-table > tbody:nth-child(1) > tr:nth-child(6) > td:nth-child(2)')[0].text
    except Exception:
        date = None
        logging.exception("This book has persian calendar!")
    return date.strip() 

In [76]:
def get_ad_date(soup):
    pass #TODO
    return None

In [77]:
def get_size(soup):
    size = soup.select(
        'html body div.container div div.product-container.well.clearfix div.clearfix div.col-md-9.col-sm-9 div.row div.col-md-5 table.product-table tbody tr td.rtl')[0].text
    return size.strip()


In [78]:
def get_material(soup):
    pass #TODO
    return None


In [79]:
def get_series(soup):
    pass #TODO
    return None


In [80]:
def get_send_time(soup):
    pass
    return None


In [81]:
def get_summary(soup):
    try:
        summary = soup.select('.product-description')[0].text.strip()
    except Exception:
        summary = None
        logging.exception("This book has no summary!")
    return summary

In [82]:
def get_tags(soup):
    tags = soup.select('.product-tags-item')
    tags_list = []
    for tag in tags:
        tags_list += [tag.text.strip()]
    return tags_list

In [87]:
 def get_book_detail(book_soup):
    book_fa_title = get_fa_title(book_soup)
    book_en_title = get_en_title(book_soup)
    book_price = get_price(book_soup)
    book_discount_percent = get_discount(book_soup)
    book_score = get_score(book_soup)
    book_publisher = get_publisher(book_soup)
    book_author = get_author(book_soup)
    book_code = get_code(book_soup)
    book_Isbn = get_isbn(book_soup)
    book_pages = get_pages(book_soup)
    book_publication_per_date = get_per_date(book_soup)
    book_publication_ad_date = get_ad_date(book_soup)
    book_size = get_size(book_soup)
    book_cover_material = get_material(book_soup)
    book_print_series = get_series(book_soup)
    book_earliest_send_time = get_send_time(book_soup)
    book_summary = get_summary(book_soup)
    book_tags = get_tags(book_soup)
    
    book_data = [book_fa_title, book_en_title, book_price, book_discount_percent, book_score, book_publisher, book_author,
            book_code, book_Isbn, book_pages, book_publication_per_date, book_publication_ad_date, book_size,
            book_cover_material, book_print_series, book_earliest_send_time, book_summary, book_tags]  #TODO
    return book_data

In [88]:
data_list = []
links = get_links()[:200]

for link in links:
    try:
        site_soup = get_soup(link)
        data = get_book_detail(site_soup)
        data_list += [data];
    except Exception:
        logging.exception("An error occurred");

ERROR:root:This book has no summary!
Traceback (most recent call last):
  File "C:\Users\raeim\AppData\Local\Temp\ipykernel_7964\1115907846.py", line 3, in get_summary
    summary = soup.select('.product-description')[0].text.strip()
IndexError: list index out of range
ERROR:root:This book has no summary!
Traceback (most recent call last):
  File "C:\Users\raeim\AppData\Local\Temp\ipykernel_7964\1115907846.py", line 3, in get_summary
    summary = soup.select('.product-description')[0].text.strip()
IndexError: list index out of range
ERROR:root:This book has no summary!
Traceback (most recent call last):
  File "C:\Users\raeim\AppData\Local\Temp\ipykernel_7964\1115907846.py", line 3, in get_summary
    summary = soup.select('.product-description')[0].text.strip()
IndexError: list index out of range
ERROR:root:This book has no summary!
Traceback (most recent call last):
  File "C:\Users\raeim\AppData\Local\Temp\ipykernel_7964\1115907846.py", line 3, in get_summary
    summary = soup.sel

In [89]:
tableOfData = pd.DataFrame(data_list,
                           columns=['fa_title', 'en_title', 'price', 'discount', 'score', 'publisher', 'author',
                                    'code', 'Isbn', 'pages', 'publication_per_date', 'publication_ad_date', 'size',
                                    'cover_material', 'print_series', 'earliest_send_time', 'summary', 'tags'])
tableOfData

,fa_title,en_title,price,discount,score,publisher,author,code,Isbn,pages,publication_per_date,publication_ad_date,size,cover_material,print_series,earliest_send_time,summary,tags
0,کتاب کلیات پزشکی سنتی چینی و طب سوزنی,Pezeshki Sonnati,300000,15.0,3.87,المعی,حسن رضوانی,70554,978-9647553261,1402,شومیز,None,وزیری,None,None,None,طب چینی یا سوزنی یکی از دو رشته درمانی طبی دنی...,"[پزشکی, ادبیات ایران, کتاب مصور, بهداشت]"
1,کتاب بانوان عمارت میسالونگی,The Ladies of Missalonghi,30000,15.0,3.17,روشنگران و مطالعات زنان,کالین مک کالو,51563,978-9641940289,184,1388,None,رقعی,None,None,None,None,"[ادبیات استرالیا, داستان تاریخی, داستان عاشقان..."
2,کتاب گوزن شاخدار فایده اش چیه؟,What Use Is A Moose?,70000,15.0,3.58,علمی و فرهنگی,مارتین وادل,49138,978-9644458071,32,1400,None,وزیری,None,None,None,None,"[ادبیات ایرلند, ادبیات داستانی, دهه 1990 میلاد..."
3,کتاب تانیا,Tanya,75000,20.0,3.62,پژواک فرزان,آلکسی آربوزوف,116421,978-6009313884,159,1396,None,جیبی,None,None,None,نمایش‌نامه «تانیا» داستان زندگی زنی جوان است ک...,"[ادبیات کلاسیک, ادبیات روسیه, دهه 1930 میلادی,..."
4,کتاب دانه های روشنایی,Flakes of light,9000,30.0,3.67,اشک,عباس مهرپویا,76016,978-9642683864,1388,شومیز,None,خشتی,None,None,None,None,"[داستان فانتزی, ادبیات داستانی, ادبیات کودک و ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,کتاب آینده پسرت را بساز,How to Raise a Boy,195000,15.0,3.44,کتاب کوله پشتی,مایکل سی ریچرت,39335,978-6004613972,344,1402,None,رقعی,None,None,None,در زمانی که بسیاری از پسران در بحران هستند ، ی...,"[ادبیات آمریکا, دهه 2010 میلادی, روانشناسی, تع..."
184,کتاب این کتاب عکس ندارد!,The Book with No Pictures,49000,20.0,3.84,پرتقال,بی جی نواک,19734,978-6004623056,48,1400,None,رحلی,None,None,None,هشدار!خب بله، این کتاب بسیار بسیار بسیاااااااا...,"[ادبیات آمریکا, ادبیات داستانی, ادبیات کودک و ..."
185,کتاب مصاحبه,Interview,40000,15.0,3.98,نیلا,محمد رحمانیان,40085,978-9648564952,1399,شومیز,None,رقعی,None,None,None,None,"[ادبیات نمایشی, نمایشنامه درام, ادبیات ایران, ..."
186,کتاب جهان باکتری ها,The Surprising World of Bacteria with Max Axi...,35000,20.0,3.3,آوای روزان,انیسکا بسکاپ,34264,978-6008267355,32,1395,None,وزیری,None,None,None,با ابر دانشمند مکس آکسیوم، دیدار کنید او توانا...,"[ادبیات آمریکا, ادبیات کودک و نوجوان, ادبیات و..."


In [26]:
file_path = "data.csv"

tableOfData.to_csv(file_path, index=False, encoding='utf-8')